# QuakeMigrate example - Icequake detection in Iceland

## Overview:

This notebook shows how to run QuakeMigrate for icequake detection, using a 2 minute window of continuous seismic data from Hudson et al. (2019). Please refer to this paper for details and justification of the settings used.

Here, we detail how to:
1. Create travel-time lookup tables for the example seismometer network
2. Run the detect stage to coalesce energy through time
3. Run the trigger stage to determine events above a threshold value
4. Run the locate stage to refine the earthquake location

We also include an outline of some of the key outputs.

In [ ]:
# Import necessary modules:
from pyproj import Proj

import pandas as pd

import QMigrate.io.data as qdata
import QMigrate.io.quakeio as qio
import QMigrate.lut.lut as qlut
import QMigrate.signal.onset.staltaonset as qonset
import QMigrate.signal.pick.gaussianpicker as qpick
import QMigrate.signal.scan as qscan
import QMigrate.signal.trigger as qtrigger

In [ ]:
# Set i/o paths:
station_file = "./inputs/iceland_stations.txt"
data_in   = "./inputs/mSEED"
lut_out   = "./outputs/lut/icequake.LUT"
out_path  = "./outputs/runs"
run_name  = "icequake_example"

## Create travel-time lookup tables (LUT)

In [ ]:
import QMigrate.lut.create_lut as clut

# Read in station information
stations = qio.stations(station_file)

# Define projections
cproj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gproj = Proj("+proj=lcc +lon_0=-17.224 +lat_0=64.328 +lat_1=64.32 +lat_2=64.335 +datum=WGS84 +units=m +no_defs")

# Set the parameters for the travel-times lookup table (LUT)
# Cell count (x,y,z); cell size (x,y,z in metres)
lut = qlut.LUT(ll_corner=[-17.24363934275664, 64.31947715407385, -1390.],
               ur_corner=[-17.204348515198255, 64.3365202025144, 1390],
               cell_size=[100., 100., 20.], grid_proj=gproj, coord_proj=cproj)

# Compute for a homogeneous velocity model
vp = 3630
vs = 1833
clut.compute(lut, stations, method="homogeneous", vp=vp, vs=vs)

# Save the LUT
lut.save(lut_out)

## Coalesce the seismic energy through time

In [ ]:
# Create a new instance of the MSEED class and set path structure
data = qdata.Archive(station_file=station_file, archive_path=data_in)
data.path_structure(archive_format="YEAR/JD/*_STATION_*")

# Create a new instance of Onset object
onset = qonset.ClassicSTALTAOnset()
onset.p_bp_filter = [10, 125, 4]
onset.s_bp_filter = [10, 125, 4]
onset.p_onset_win = [0.01, 0.25]
onset.s_onset_win = [0.05, 0.5]

# Create a new instance of the SeisScan class
scan = qscan.QuakeScan(data, lut, onset=onset, output_path=out_path, run_name=run_name)

# Set detect parameters
scan.sampling_rate = 500
scan.time_step = 0.75
scan.n_cores = 12

# Defining the start and end times 
starttime = "2014-06-29T18:41:55.0"
endtime   = "2014-06-29T18:42:20.0"

In [ ]:
# Run the detect stage to find the coalescence of energy through time:
scan.detect(starttime, endtime)

## Run the trigger stage, to detect and output individual icequakes

nb: We can use the same QuakeScan object here because we are not using a different decimation. If running trigger and locate on grids with different levels of decimation, a new QuakeScan object should be created.

In [ ]:
trig = qtrigger.Trigger(out_path, run_name, stations)

trig.normalise_coalescence = True
trig.marginal_window = 2.75
trig.minimum_repeat = 6.
trig.detection_threshold = 1.8

# Run trigger
trig.trigger(starttime, endtime, savefig=True)

## Run the locate stage, to relocate triggered events on a less decimated grid

In [ ]:
# Create a new instance of PhasePicker object
picker = qpick.GaussianPicker(onset=onset)
picker.marginal_window = 2.75
picker.plot_phase_picks = True

# Create a new instance of QuakeScan object
scan = qscan.QuakeScan(data, lut, onset=onset, picker=picker,
                       output_path=out_path, run_name=run_name, log=True)

# Set locate parameters:
scan.sampling_rate = 500
scan.marginal_window = 2.75
scan.n_cores = 12

# Turn on plotting features
scan.plot_event_summary = True
scan.plot_event_video = False
scan.write_cut_waveforms = False

In [ ]:
# Run the locate stage to determine the location of any triggered events
scan.locate(start_time=starttime, end_time=endtime)

## Some of the key outputs

In [ ]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = "./outputs/runs/icequake_example/events/20140629184210336000.event"
event_df = pd.read_csv(icequake_event_fname)

event_df

In [ ]:
# Show the .picks file, containing station time picks:
icequake_pick_fname = "outputs/runs/icequake_example/picks/20140629184210336000.picks"
pick_df = pd.read_csv(icequake_pick_fname)

pick_df

In [ ]:
# Show the coalescence pdf file, containing event origin time and location:
icequake_coal_image_fname = "outputs/runs/icequake_example/summaries/icequake_example_20140629184210336000_EventSummary.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_coal_image_fname, width=800, height=400) # Plot pdf

References:

Hudson, T.S., Smith, J., Brisbourne, A.M., and White R.S. (2019). Automated detection of basal icequakes and discrimination from surface crevassing. Annals of Glaciology, 79